In [ ]:
%%capture
#export
from typing import *

from fastai2.basics import *
from fastai2.text.all import *

from transformers import PreTrainedTokenizer, AutoTokenizer

In [ ]:
# default_exp general

# All functions
> Now put all functions in this module. Consider split to multiple modules if we have more functions.

## Tokenizer

In [ ]:
#export
class TransformersTokenizer():
    ''' fastai want the tokenizer can handle list of string.
        use in parallel_gen() '''
    def __init__(self, tokenizer: PreTrainedTokenizer):
        self.tokenizer = tokenizer
    def __call__(self, items: List[str]):
        return map(self.tokenizer.tokenize, items)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

texts = ['This is a test', 'Just test']
transfomersTokenizer = TransformersTokenizer(tokenizer)
tok_texts = list(transfomersTokenizer(texts))

test_eq(tok_texts, [['this', 'is', 'a', 'test'], ['just', 'test']])

In [ ]:
texts = ['This is a test', 'Just test']
# parallel_gen will return generator of (0, ['this', 'is', 'a', 'test']), (1, ['just', 'test'])
tok_texts = L(parallel_gen(TransformersTokenizer, texts, tokenizer=tokenizer)).itemgot(1)

test_eq(tok_texts, [['this', 'is', 'a', 'test'],['just', 'test']])

## Transform

In [ ]:
#export
class TransformersNumericalize(Transform):
    def __init__(self, tokenizer: PreTrainedTokenizer):
        self.tokenizer = tokenizer
    def encodes(self, o):
        ''' o: list of string token, returns: tensored list of int token '''
        return TensorText(
            self.tokenizer.build_inputs_with_special_tokens(
                self.tokenizer.convert_tokens_to_ids(o)
            )
        )
    def decodes(self, o):
        return TitledStr(self.tokenizer.decode(o))

In [ ]:
tok_list = ['this', 'is', 'a', 'test']
num_list = TensorText([ 101, 2023, 2003, 1037, 3231,  102])
transformersNumericalizer = TransformersNumericalize(tokenizer)

test_eq(transformersNumericalizer.encodes(tok_list), num_list)
test_eq(transformersNumericalizer.decodes(num_list), '[CLS] this is a test [SEP]')

In [ ]:
#export
class Pad2Max(Transform):
    ''' pad rank one tensor by pad_idx to max_len, if original len is larger than max_len, truncate it
    '''
    def __init__(self, max_len, pad_idx):
        self.max_len = max_len
        self.pad_idx = pad_idx
    def encodes(self, o):
        ori_len = len(o)
        result = o
        if ori_len > self.max_len:
            result = o[:self.max_len]
        elif ori_len < self.max_len:
            result = nn.functional.pad(o, [0, self.max_len-ori_len], value=self.pad_idx)
        assert len(result) == self.max_len, f'len(o): {len(o)}, max_len: {self.max_len}'
        return result
    def decodes(self, o):
        return TensorText([v for v in o if v != self.pad_idx])

In [ ]:
pad2max = Pad2Max(10, 1)

num_list = torch.tensor([ 101, 2023, 2003, 1037, 3231,  102])
padded_num_list = torch.tensor([ 101, 2023, 2003, 1037, 3231,  102,    1,    1,    1,    1])
test_eq(pad2max(num_list), padded_num_list)

pad2max = Pad2Max(3, 1)
num_list = torch.tensor([ 101, 2023, 2003, 1037, 3231,  102])
padded_num_list = torch.tensor([ 101, 2023, 2003])
test_eq(pad2max(num_list), padded_num_list)

pad2max = Pad2Max(6, 1)
num_list = torch.tensor([ 101, 2023, 2003, 1037, 3231,  102])
padded_num_list = torch.tensor([ 101, 2023, 2003, 1037, 3231,  102])
test_eq(pad2max(num_list), padded_num_list)

## Splitter

In [ ]:
#export
''' Print the model, look at its architecture, then write down the split '''
def bert_SeqClassification_split(m:nn.Module): 
    # 12 layers, 110M params
    return L(m.bert.embeddings, *m.bert.encoder.layer, m.bert.pooler, m.classifier).map(params)
def distilbert_SeqClassification_split(m:nn.Module): 
    # 6 layers, 66M params
    return L(m.distilbert.embeddings, *m.distilbert.transformer.layer, m.pre_classifier, m.classifier).map(params)
def albert_SeqClassification_split(m: nn.Module):
    return L(m.albert.embeddings, *m.albert.encoder.albert_layer_groups, m.albert.pooler, m.classifier).map(params)
def roberta_SeqClassification_split(m:nn.Module): 
    return L(m.roberta.embeddings, *m.roberta.encoder.layer, m.roberta.pooler, m.classifier).map(params)

def gpt2_lmhead_split(m:nn.Module): 
    # 12-layer, 768-hidden, 12-heads, 117M parameters.
    return L(m.transformer.wte, m.transformer.wpe, *m.transformer.h, m.lm_head).map(params)

## Callback

In [ ]:
class FakeLearner():
    def __init__(self, cb, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)
        cb.learn = self
        self.cb = cb
    
    def run_cb(self, event_name):
        getattr(self.cb, event_name)()

In [ ]:
#export
class GPT2LMHeadCallback(Callback):
    '''The output of AutoModelWithLMHead is (last_hidden_state, past)
        What fastai want is last_hidden_state '''
    def after_pred(self):
        last_hidden_state = self.learn.pred[0]
        self.learn.pred = last_hidden_state

In [ ]:
learn = FakeLearner(cb=GPT2LMHeadCallback(), pred=('last_hidden_state', 'past'))
learn.run_cb('after_pred')
test_eq(learn.pred, 'last_hidden_state')

In [ ]:
#export
class BertSeqClassificationCallback(Callback):
    ''' It should be ok to use it in all Bert like model. eg: Roberta
    '''
    def __init__(self, pad_id: int):
        self.pad_id = pad_id
    
    def begin_batch(self):
        ''' Instead of input_ids, we need to pass the attention_mask to AutoModelForSequenceClassification to avoid it to attention to padding tokens.
        '''
        input_ids = self.learn.xb[0]
        device = input_ids.device
        attention_mask = torch.where(input_ids == self.pad_id, torch.tensor(0, device=device), torch.tensor(1, device=device)).to(input_ids)
        self.learn.xb = [input_ids, attention_mask]
    
    def after_pred(self):
        ''' The output of AutoModelForSequenceClassification is (logits, )
            What fastai want is logits '''
        logits = self.learn.pred[0]
        self.learn.pred = logits

In [ ]:
input_ids = torch.tensor([[4, 3, 1, 1], 
                          [5, 6, 7, 1]])
attention_mask = torch.tensor([[1, 1, 0, 0], 
                               [1, 1, 1, 0]])

learn = FakeLearner(cb=BertSeqClassificationCallback(pad_id=1), xb=(input_ids,))
learn.run_cb('begin_batch')
test_eq(learn.xb, (input_ids, attention_mask))

learn = FakeLearner(cb=BertSeqClassificationCallback(pad_id=1), pred=('logits',))
learn.run_cb('after_pred')
test_eq(learn.pred, 'logits')

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_general.ipynb.
Converted 01_example_roberta_classification.ipynb.
Converted 02_example_gpt2_lm.ipynb.
Converted index.ipynb.
